# Psycholinguistic classifier

In [1]:
%load_ext autoreload
%autoreload 2
from data import Data
import pandas as pd

In [2]:
df = pd.read_csv('utils/Psycholinguistic_Features.txt', sep='\t')
df_mean = df.mean(axis=0)
training_data = ['datasets/english/News_Train.tsv',
                 'datasets/english/WikiNews_Train.tsv',
                 'datasets/english/Wikipedia_Train.tsv',
                 ]
data = Data()
data.load_data(training_data)

In [3]:
def get_psicolinguistics(df, df_mean, words):
    psicos = []
    for word in words:
        psico = {'Familiarity': 0, 'Age_of_Acquisition':0, 'Concreteness':0, 'Imagery': 0}
        infos = df[df.Word == word]
        if not infos.empty:
            psico['Familiarity'] += infos['Familiarity'].values[0]
            psico['Age_of_Acquisition'] += infos['Age_of_Acquisition'].values[0]
            psico['Concreteness'] += infos['Concreteness'].values[0]
            psico['Imagery'] += infos['Imagery'].values[0]
        else:
            psico['Familiarity'] += df_mean['Familiarity']
            psico['Age_of_Acquisition'] += df_mean['Age_of_Acquisition']
            psico['Concreteness'] += df_mean['Concreteness']
            psico['Imagery'] += df_mean['Imagery']
        psicos.append(pd.Series(psico, index=['Familiarity', 'Age_of_Acquisition', 'Concreteness', 'Imagery']))
    return pd.DataFrame(psicos).mean()

In [4]:
get_psicolinguistics(df, df_mean, ['house', 'bla'])

Familiarity           453.618040
Age_of_Acquisition    369.559318
Concreteness          473.870993
Imagery               461.919200
dtype: float64

-----
# Tests

* target words not in psycholinguistic_features

In [ ]:
count = 0
errors = []
for instance in data.instances:
    for target in instance.target:
        try:
            if len(df[df.Word == instance.tokens[target]].index) == 0:
                count += 1
                errors.append(instance.tokens[target])
        except:
            print(target, instance.target_chars, instance.tokens)
print('%d / %d' % (count, len(data.instances)))
print(list(set(errors)))